# 7-Slot Nerdle Solver Test
Nerdle game with intermediate problem size (between mini and full). The best start has all different numbers: `28/7=4`, which needs at most $X$ guesses and $X \pm X$ guesses.

In [60]:
%load_ext autoreload
%autoreload 2

import collections
import itertools
import numpy as np
import matplotlib.pyplot as plt

import nerdle
import score as s
import generator
from nerdle import Hint, NerdleData

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [61]:
# Mini-Nerdle.
NUM_SLOTS = 7
SCORE_DB_FILE = "db/nerdle{}_matrix.db".format(NUM_SLOTS) 

solver_data = nerdle.create_solver_data(NUM_SLOTS, SCORE_DB_FILE, strategy="matrix")

0 / 6661 (0.0%) completed
333 / 6661 (5.0%) completed
666 / 6661 (10.0%) completed
999 / 6661 (15.0%) completed
1332 / 6661 (20.0%) completed
1665 / 6661 (25.0%) completed
1998 / 6661 (30.0%) completed
2331 / 6661 (35.0%) completed
2664 / 6661 (40.0%) completed
2997 / 6661 (45.0%) completed
3330 / 6661 (50.0%) completed
3663 / 6661 (55.0%) completed
3996 / 6661 (60.0%) completed
4329 / 6661 (65.0%) completed
4662 / 6661 (70.0%) completed
4995 / 6661 (75.0%) completed
5328 / 6661 (80.0%) completed
5661 / 6661 (85.0%) completed
5994 / 6661 (90.0%) completed
6327 / 6661 (95.0%) completed
6660 / 6661 (100.0%) completed


In [64]:
d = solver_data.score_db
d.shape

(6661, 6661)

## Example Usage

In [66]:
score = nerdle.score_guess("1*1*6=6", "99/9=11")
hints = [Hint.MISPLACED, Hint.INCORRECT, Hint.MISPLACED, Hint.INCORRECT, Hint.INCORRECT, Hint.MISPLACED, Hint.INCORRECT]
print(score, 
      s.score_to_hint_string(score, NUM_SLOTS),
      s.score_to_hints(score, NUM_SLOTS) == hints,
      s.hint_string_to_score(s.score_to_hint_string(score, NUM_SLOTS))
     )

2082 ?-?--?- True 2082


In [67]:
# A good initial guess significantly reduces the number of answers. In this case, from
# 206 to 10.
guess_history, hint_history, answer_size_history =  nerdle.NerdleSolver(solver_data).solve("99/9=11", initial_guess= "1+6-7=0", debug=True)

--> guess 1+6-7=0 guesses_left 5
score ?----?- 2050
answers 128
--> guess 2*19=38 guesses_left 4
score --?++-- 352
answers 1
--> guess 99/9=11 guesses_left 3
score +++++++ 5461


## Benchmark
This is a fast in-memory numpy array implementation.

In [70]:
%time guess_history, hint_history, answer_size_history = nerdle.NerdleSolver(solver_data).solve("99/9=11", initial_guess="1+6-7=0")

CPU times: user 259 ms, sys: 3.15 ms, total: 263 ms
Wall time: 266 ms


In [69]:
answer = "99/9=11"
solver = nerdle.NerdleSolver(solver_data)
for start in list(solver_data.answers)[:5]:
    print(start)
    %time solver.solve(answer, initial_guess=start) 

1*1*1=1
CPU times: user 136 ms, sys: 4.09 ms, total: 141 ms
Wall time: 141 ms
1*1*2=2
CPU times: user 51.7 ms, sys: 202 µs, total: 51.9 ms
Wall time: 52.1 ms
1*1*3=3
CPU times: user 57.3 ms, sys: 517 µs, total: 57.9 ms
Wall time: 57.8 ms
1*1*4=4
CPU times: user 52.9 ms, sys: 469 µs, total: 53.3 ms
Wall time: 53.8 ms
1*1*5=5
CPU times: user 48.4 ms, sys: 46 µs, total: 48.5 ms
Wall time: 48.5 ms


## Initial Guess Optimization
Assuming an initial guess with a lot of different numbers and operators is best.

In [77]:
start = "1+6-7=0"
solutions = [nerdle.NerdleSolver(solver_data).solve(answer, initial_guess=start) 
             for answer in solver_data.answers]
n = np.array([len(solution[0]) for solution in solutions])
num_answers = len(solver_data.answers)
compression_ratio = num_answers / np.array([solution[2][0] for solution in solutions])
print(np.mean(compression_ratio), np.std(compression_ratio))

fig, axs = plt.subplots(1, 2, figsize=(10, 4))

ax = axs[0]
ax.hist(n);
ax.set_title("#Guesses for start {}".format(start));

ax = axs[1]
ax.hist(compression_ratio);
ax.set_title("Compression Ratio Distribution, start {}".format(start));

KeyboardInterrupt: 

## Profiling

In [72]:
import cProfile
import pstats
from pstats import SortKey

In [75]:
cProfile.run('guess_history, hint_history, answer_size_history = nerdle.NerdleSolver(solver_data).solve("99/9=11", initial_guess="1+6-7=0", debug=True)', 'stats')
p = pstats.Stats('stats')
p.sort_stats(SortKey.CUMULATIVE).print_stats(20);

--> guess 1+6-7=0 guesses_left 5
score ?----?- 2050
answers 128
--> guess 2*19=38 guesses_left 4
score --?++-- 352
answers 1
--> guess 99/9=11 guesses_left 3
score +++++++ 5461
Wed Sep 14 13:59:59 2022    stats

         133518 function calls (133497 primitive calls) in 0.280 seconds

   Ordered by: cumulative time
   List reduced from 60 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.280    0.280 {built-in method builtins.exec}
        1    0.000    0.000    0.280    0.280 <string>:1(<module>)
        1    0.000    0.000    0.280    0.280 /Users/olivne/oren/nerdle-solver/nerdle.py:190(solve)
        1    0.000    0.000    0.280    0.280 /Users/olivne/oren/nerdle-solver/nerdle.py:195(solve_adversary)
        2    0.000    0.000    0.279    0.140 /Users/olivne/oren/nerdle-solver/nerdle.py:226(make_guess)
        2    0.004    0.002    0.266    0.133 {built-in method builtins.min}
    13324    0.093 

Looks like Cython gives 2x speedup for the core function `score_guess`:

In [76]:
import score_guess as sg
cProfile.run('for _ in range(200000): sg.score_guess("1*1*6=6", "99/9=11")', 'stats')
p = pstats.Stats('stats')
p.sort_stats(SortKey.CUMULATIVE).print_stats(20)

Wed Sep 14 14:00:02 2022    stats

         200003 function calls in 0.553 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.553    0.553 {built-in method builtins.exec}
        1    0.043    0.043    0.553    0.553 <string>:1(<module>)
   200000    0.510    0.000    0.510    0.000 {score_guess.score_guess}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}


